In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import skew, kurtosis, kurtosistest, norm, t

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import bokeh
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.palettes import inferno
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Legend, Span, Label
output_notebook()

import tqdm

Hello everyone,

The cryptocurrency market has experienced last month one of its largest corrections of recent history, resulting in a drop of approximately 57.8% in total market capitalization (reducing from 829.5 bn in January 8th to 349.5 bn in only 30 days, according to https://coinmarketcap.com).

In [ ]:
data = pd.read_csv("../input/crypto-markets.csv")
marketcap_df = pd.pivot_table(data, index='date', columns='symbol', values='market',
                              fill_value=0.0).loc['2017-01-01':]

plt.figure(figsize=(13,6))
marketcap_df.sum(axis=1).divide(1000000000).plot()
plt.title("Market Capitalization")
plt.ylabel("value ($ bn)")
plt.xlabel("date")
plt.grid(linestyle='--', alpha=0.5);

This strong correction is most problably explained by the sequence of negative news related to the crypto market. China temporarily banned trading of crypto assets in september; South Korea approved more severe laws and regulations against cryptocurrency exchanges; in an interview, Indian minister of finance strongly opposed against the use of cryptocurrencies as payment system; also there was a new hack case reported by the japanese exchange Coincheck. All these events together, with the speculative nature of this market resulted in the strong bearsish trend we have seen recently.

Risk management is one of the most important factors in developing a good trading strategy, specially when the market presents a strong downtrend. The main idea of this article is to show, in a simple way, some efficient techniques to reduce the risk from a investment portfolio. Combining risk aversion techniques with the trading strategy results in a more robust portfolio and less susceptible to market oscillations. It is important to emphasize that in high-risk markets where the assets are heavily correlated, like the cryptocurrencies, the risk is intrinsic to the market and without proper diversification of investments it is impossible to avoid completely major market corrections.  

In turbulent times like this, it is up to the trading strategy to assume the lowest risk possible given market limitations. Some techniques to reduce risk will be presented next, in order to make clear to investors how we position our funds when the market presents downtrend characteristics.
   
### Universe Selection

One of the most important decisions in any trading strategy is to choose which assets should be included as investiment options. At this moment, there are more than one hundred different coins/tokens available for trading in our algorithm. However, the right selection of these assets not only improve the strategy's perfomance but significantly reduce the risk. We carefully select every week the best 20 assets according to their preponderance (market value, liquidity, transactions volume, price spread), in order to keep only the coins that are relevant as investment options and avoid portfolios where the assets present decaying performance. In fact, currently the 20 biggest crypto assets represent 87.5% of the value of all market as we see below.
 

In [ ]:
marketcap_df = marketcap_df.apply(lambda x: [x[i] / x.sum() for i in range(x.shape[0])],
                                  axis=1).astype('f')

smoothed_mc = marketcap_df.ewm(alpha=0.1).mean()

n = 20
largest = pd.DataFrame(index=marketcap_df.columns)
for i, row in enumerate(smoothed_mc.iterrows()):
    # choose pairs every quarter
    if i % int(7) == 0:
        components_index = np.argpartition(row[1].values, -n)[-n:]
    largest[row[0]] = row[1].iloc[components_index].fillna(0)
largest = largest.T
pie = largest.iloc[-1].dropna().sort_values()
pie['Other Assets'] = 1 - largest.iloc[-1].sum()
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.pie(pie.values, labels = pie.index, radius=1.5, autopct='%.1f %%', pctdistance=0.8, shadow=True, explode=np.append(np.zeros(len(pie)-1),0.2))
ax1.axis('equal');

  
### Portfolio Diversification

Diversification is an efficient form of reducing risk in any investment. An investment portfolio with a variety of uncorrelated assets is considerably less susceptible to large oscillations, since the chance of all assets present extreme performance simutaneously is low. In practice, profits and losses of different assets tend to oppose to each other, and as result, oscillations on the value of the portfolio are considerably diminished. Please note that the crypto market still present great correlation between all assets, this fact wakeans the effect of diversification as a measure to reduce risk. However, all the coins and tokens with different characteristics and applications available in this market creates a trend of increasing independence between them, which consequently tend to decorrelate these assets.


### Rebalanceamento

Rebalanceamento do portifolio é uma estratégia que ajuda o investidor a manter uma exposição constante ao universo de ativos a fim de extrair máxima performance com relaçao ao mercado. É o processo de reajustar o portifolio de forma a manter sempre uma alocação alvo. Por exemplo, digamos que uma estratégia de alocação indica que deve-se alocar 60% do portifolio no ativo A e 40% no ativo B e manter a posição constante; mesmo que depois de alguns dias o ativo A tenha apresentado melhor performance, digamos que agora representa 65% do seu portifolio contra 35% no ativo B, o rebalanceamento te leva a tirar investimento de A e mover para B até retornar ao alvo de 60%/40%. Rebalanceamento é uma estratégia de "compre na baixa / venda na alta", mesmo que pareça contra-intuitivo é uma estratégia altamente eficaz especialmente quando o mercado não apresenta fortes indícios de tendência.

### Indicadores de Risco

Avaliar um tipo de investimento apenas através do seu retorno passado não é a melhor alternativa. Felizmente, existem diferentes indicadores na literatura que representam a chance do retorno de um investimento ser diferente do inicialmente esperado, o que inclui a possibilidade de perder parte ou todo o valor aplicado. Indicadores de risco são resultados de cálculos feitos em dados históricos com o objetivo de quantificar o risco assumido para determinada posição de investimento. Por exemplo, um dos indicadores mais comuns usados por investidores é o Value-at-Risk (VaR), que é uma medida que indica a perda máxima, com certo grau de confiança, proveniente de oscilações de preços de mercado medidos dentro de um período determinado. Em outras palavras, o VaR(5%) representa a perda máxima potencial para o próximo período, com um grau de confiança de 95%. Existem diversos outros indicadores de risco que são especialmente relevantes e devem ser recalculados frequentemente a fim de manter um investimento com risco aceitável.


Para completar este artigo, mostraremos o cálculo do VaR para uma sequência de retornos. Utilizaremos dados históricos do preço da Bitcoin como exemplo. Os dados sao cortesia de https://coinmarketcap.com.

In [ ]:
close_df = pd.pivot_table(data, index='date', columns='symbol', values='close',
                              fill_value=0.0).loc['2015-01-01':]

Para começar, adquirimos os dados de retornos da Bitcoin de janeiro de 2015 até fevereiro de 2018, sendo estes as diferenças percentuais entre preços adjascentes. Assim:

In [ ]:
price = close_df.BTC.astype('f').dropna()
ret = price.pct_change().dropna()

plt.figure(figsize=(12,4))
plt.title("Preço da Bitcoin em USD")
plt.ylabel("preço")
plt.plot(pd.to_datetime(price.index), price)

plt.figure(figsize=(12,4))
plt.title("Retornos da Bitcoin em %")
plt.ylabel("retornos")
plt.plot(pd.to_datetime(ret.index), ret);

Desta sequência de retornos, podemos estimar uma distribuição empírica de probabilidades. Fazemos isto contando o número de ocorrências para cada magnitude. A frequência das ocorrencias é mostrada abaixo em um histograma.

In [ ]:
grey = .66, .66, .77
plt.figure(figsize=(13,6))
plt.title("Distribuição de retornos")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.hist(ret, bins=50, normed=True, color=grey, edgecolor='none');

Com isto, já é possível estimar o VaR empírico da distribuição. Este é definido como o valor associado a um certo nível de confiança ($ alpha $), dada distribuição.
Para fazer isso, organizamos as amostras da menor para a maior e escolhemos a amostra que bate, em posição, com nosso intervalo de confiança, definido como $ 1 - alpha $.

In [ ]:
alpha1 = 0.05
alpha2 = 0.01

sorted_ret = np.sort(ret)

mu = ret.mean()
print("Média da distribuição: %.4f %%" % 
        mu
 )

var95 = abs(sorted_ret[int((1 - alpha1) * sorted_ret.shape[0])])
print("%.d%% Empirical VaR: %.2f %%" % ((
    1 - alpha1) * 100,
    var95 * 100)
 )

var99 = abs(sorted_ret[int((1 - alpha2) * sorted_ret.shape[0])])
print("%.d%% Empirical VaR: %.2f %%" % ((
    1 - alpha2) * 100,
    var99 * 100)
 )

grey = .66, .66, .77
plt.figure(figsize=(13,6))
plt.title("Distribuição de retornos")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.hist(ret, bins=50, normed=True, color=grey, edgecolor='none');
plt.text(mu+0.003, 23, "Média: %.4f" % mu, color='black')
plt.plot([mu, mu], [0, 24], c='black')
plt.plot([-var95, -var95], [0, 6], c='b')
plt.text(-var95-0.01, 7.5, "95%% VaR", color='b')
plt.text(-var95-0.01, 6.5, var95, color='b')
plt.plot([-var99, -var99], [0, 4], c='r')
plt.text(-var99-0.01, 5.5, "99%% VaR", color='r')
plt.text(-var99-0.01, 4.5, var99, color='r');
plt.grid(linestyle='--')

Fazendo desta forma, não utilizamos de nenhum modelo hipotético, nem adicionamos nenhum conhecimento apriorístico a análise. Isso pode, ou não, ser vantajoso, dependendo de quão bem o modelo escolhido se ajusta aos dados. Na maioria dos casos, o cálculo do VaR é realizado assumindo-se uma distribuição normal de retornos. A distribuição normal assume caldas que decaem exponencialmente, e portanto não concentra muita probabilidade nas regiões mais distantes da média.

In [ ]:
dx = 0.0001  # resolution
x = np.arange(-5, 5, dx)
pdf = norm.pdf(x, 0, 1)

plt.figure(figsize=(13, 5))
plt.plot(x, pdf, 'b')
plt.title("Distribuição normal")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.text(0.05, 0.42, "Média", color='black')
plt.plot([0, 0], [0, 0.43], c='black')
cred = norm.ppf(1-0.01) * 1 - 0
plt.plot([-cred, -cred], [0, 0.11], c='r')
plt.text(-cred-0.9, 0.1, "95%% cred", color='r')
plt.grid(linestyle='--')

Vamos prametrizar uma distribuição normal utilizando os dados de retornos da Bitcoin e analisar o resultado obtido para o VaR utilizando este modelo.

In [ ]:
mu_norm, sig_norm = norm.fit(ret)
dx = 0.0001  # resolution
x = np.arange(-0.2, 0.2, dx)
pdf = norm.pdf(x, mu_norm, sig_norm)

var95 = norm.ppf(1 - alpha1) * sig_norm - mu_norm
var99 = norm.ppf(1 - alpha2) * sig_norm - mu_norm

print("%.d%% Normal VaR: %.2f %%" % ((
    1 - alpha1) * 100,
    var95 * 100)
     )
print("%.d%% Normal VaR: %.2f %%" % ((
    1 - alpha2) * 100,
    var99 * 100)
     )

plt.figure(figsize=(13, 5))
plt.plot(x, pdf, 'b')
plt.title("Distribuição normal")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.text(mu_norm+0.003, 10.5, "Média: %.4f" % mu_norm, color='black')
plt.plot([mu_norm, mu_norm], [0, 11], c='black')
plt.plot([-var95, -var95], [0, 4.2], c='r')
plt.text(-var95-0.035, 4.5, "95%% VaR", color='r');
plt.text(-var95-0.025, 4, "%.4f" % var95, color='r');
plt.plot([-var99, -var99], [0, 2.2], c='r')
plt.text(-var99-0.035, 2.5, "99%% VaR", color='r');
plt.text(-var99-0.025, 2, "%.4f" % var99, color='r');
plt.grid(linestyle='--')

Observa-se que os valores obtidos para o VaR utilizando a distribuição normal são menores do que aqueles obtidos utilizando-se a distribuição empírica. Isto ocorre porque a distribuição normal não é um bom modelo para retornos financeiros, principalmente para os do mercado de criptomoedas, dado que este exibe comportamentos extremos muito frequentemente. Podemos observar a falta de ajuste do modelo normal aos dados utilizando testes estatísticos. 